<a href="https://colab.research.google.com/github/Shruti022/Healthcare-Chatbot/blob/main/Copy_of_Copy_of_LLM_Project_pivot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Phase 1: Stepwise API Exploration

Step 1: Import Libraries


In [1]:
!pip install -q requests pandas streamlit pyngrok faiss-cpu sentence-transformers numpy

import requests
import pandas as pd
import json
import hashlib
from datetime import datetime
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 130.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 141.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Secure KEY INPUT
import os
import getpass

# Securely Capture Key
# Input will be invisible. Paste key and press Enter.
key_input = getpass.getpass("🔑 Enter Gemini API Key (Invisible Input): ")

if not key_input.startswith("AIza"):
    print("⚠️ Warning: Key might be invalid (usually starts with 'AIza').")
else:
    print("✅ API Key captured securely in Environment Variable.")

# 2. Set as Environment Variable for the Session
os.environ["GEMINI_API_KEY"] = key_input

🔑 Enter Gemini API Key (Invisible Input): ··········
✅ API Key captured securely in Environment Variable.


In [4]:
%%writefile build_embeddings.py
import pandas as pd
import numpy as np
import faiss
import json
from sentence_transformers import SentenceTransformer

# === REAL PATH (from readlink) ===
BASE = "/content/drive/.shortcut-targets-by-id/1-SiVJhXHTHtDYSrPmW_0VfuP7gSTePcj/data"

# ---------------------------------------------
# Load Data
# ---------------------------------------------
df = pd.read_csv(f"{BASE}/clinical_trials_diabetes_full.csv")

df["status"] = df["status"].astype(str).str.strip().str.title()
bad_status = ["Terminated", "Withdrawn", "Suspended", "No Longer Available", "Unknown"]
df_clean = df[~df["status"].isin(bad_status)].copy()

# ---------------------------------------------
# Chunking
# ---------------------------------------------
chunks = []
chunk_map = []

for idx, row in df_clean.iterrows():
    title = str(row.get("brief_title", "")).strip()
    summary = str(row.get("brief_summary", "")).strip()

    if len(summary) < 20:
        continue

    text = f"Title: {title}\nSummary: {summary}"
    chunks.append(text)

    chunk_map.append({
        "nct_id": row["nct_id"],
        "title": title,
        "text": text,
        "status": row["status"]
    })

print(f"Created {len(chunks)} chunks.")

# ---------------------------------------------
# Embeddings
# ---------------------------------------------
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embed_model.encode(chunks, batch_size=64, show_progress_bar=True)

np.save(f"{BASE}/clinical_trials_diabetes_full_embeddings.npy", embeddings)
print("Saved clinical_trials_diabetes_full_embeddings.npy")

# ---------------------------------------------
# Save chunk map
# ---------------------------------------------
with open(f"{BASE}/clinical_trials_diabetes_full_chunk_map.json", "w") as f:
    json.dump(chunk_map, f)

print("Saved clinical_trials_diabetes_full_chunk_map.json")

# ---------------------------------------------
# Build & Save FAISS
# ---------------------------------------------
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype("float32"))
faiss.write_index(index, f"{BASE}/clinical_trials_diabetes_full_faiss.index")

print("Saved clinical_trials_diabetes_full_faiss.index")
print("✅ Embedding build COMPLETE.")


Writing build_embeddings.py


In [5]:
!python build_embeddings.py

2025-11-27 06:40:14.758159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764225614.783078    1243 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764225614.802500    1243 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764225614.847084    1243 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764225614.848936    1243 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764225614.848959    1243 computation_placer.cc:177] computation placer alr

In [6]:
%%writefile utils.py
import json
import hashlib
from datetime import datetime

import faiss
from sentence_transformers import SentenceTransformer

# --- Confidence score from distance ---

def calculate_confidence_score(distance: float, normalization_factor: float = 1.0) -> float:
    """Inverse L2 distance score in (0,1]; closer = higher confidence."""
    return normalization_factor / (normalization_factor + float(distance))


# --- Load pre-built index + chunk map ---

def load_data_and_index(chunk_map_path: str, faiss_path: str):
    """Loads pre-built chunks and FAISS index for quick startup."""
    print("⏳ Loading pre-built RAG index...")

    with open(chunk_map_path, "r") as f:
        chunk_map = json.load(f)

    index = faiss.read_index(faiss_path)

    embed_model = SentenceTransformer("all-MiniLM-L6-v2")

    print(f"✅ RAG Index Ready: {index.ntotal} vectors loaded.")
    return embed_model, index, chunk_map


# --- Provenance logging ---

def log_provenance_step(agent_name: str, input_data, output_data, detail=None):
    """
    Creates a detailed log entry for a single agent step.
    """
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "agent": agent_name,
        "input": input_data,
        "output": output_data,
        "detail": detail or {},
        "model_version": "gemini-2.0-flash",
    }
    return log_entry


# --- Reproducibility hash ---

def generate_reproducibility_hash(conversation_history, corpus_version: str = "v1.0"):
    """
    Generates a deterministic session hash based on the conversation history.
    """
    queries = [turn.get("query", "") for turn in conversation_history]
    raw = f"{corpus_version}|{'|'.join(queries)}"
    return hashlib.md5(raw.encode("utf-8")).hexdigest()


Writing utils.py


In [23]:
%%writefile run_bot.py
import json
import re
import os
import sys
from typing import List, Dict, Any

import numpy as np
import requests
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# --- Updated Import: Robust Cross-Encoder Initialization ---
CrossEncoder = None
try:
    from sentence_transformers import CrossEncoder
    print("✅ sentence_transformers imported successfully.")
except ImportError:
    print("⚠️ sentence_transformers not found. Reranking will be disabled.")
except Exception as e:
    print(f"⚠️ Error importing CrossEncoder: {e}. Reranking disabled.")

from utils import (
    load_data_and_index,
    log_provenance_step,
    generate_reproducibility_hash,
    calculate_confidence_score,
)


# --- NEW CONFIG (SECURE & 2.0 MODEL) ---
API_KEY = os.environ.get("GEMINI_API_KEY")

if not API_KEY:
    print("❌ ERROR: API Key not found. Please run the 'Secure Input' cell first.")
    sys.exit(1)

genai.configure(api_key=API_KEY)

# Using the Experimental 2.0 Flash endpoint
gemini_model = genai.GenerativeModel("models/gemini-2.0-flash")

CHUNK_PATH = "/content/drive/.shortcut-targets-by-id/1-SiVJhXHTHtDYSrPmW_0VfuP7gSTePcj/data/clinical_trials_diabetes_full_chunk_map.json"
FAISS_PATH = "/content/drive/.shortcut-targets-by-id/1-SiVJhXHTHtDYSrPmW_0VfuP7gSTePcj/data/clinical_trials_diabetes_full_faiss.index"

# Load embedding model, FAISS index, and chunk metadata
embed_model, faiss_index, chunk_map = load_data_and_index(CHUNK_PATH, FAISS_PATH)

# --- NEW: Reranker Initialization ---
reranker = None
if CrossEncoder:
    try:
        print("⏳ Loading Reranker Model (Cross-Encoder)...")
        # High precision reranker
        reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
        print("✅ Reranker Loaded.")
    except Exception as e:
        print(f"⚠️ Reranker model download failed (using pure FAISS): {e}")


# =========================
# PARSER
# =========================
class SymptomParser:
    def __init__(self, model):
        self.model = model

    def parse(self, text: str):
        """
        Enhanced parser for clinical trial search queries.
        Decides if user is doing trial search vs general question, etc.
        """
        prompt = (
            "You are a clinical trial search classifier for diabetes research.\n"
            "Your job is to determine if the user wants to SEARCH for trials or just learn about diabetes.\n\n"
            f'User Input: "{text}"\n\n'
            "Classification Rules:\n"
            "1. If query contains 'trial', 'study', 'research', 'clinical', or asks 'what trials', "
            "'show me trials', 'are there trials' → intent='trial_search'\n"
            "2. If user states personal info like age, conditions, medications → intent='profile_info'\n"
            "3. If asking general education questions like 'what is X?', 'how does Y work?' "
            "(WITHOUT asking about trials) → intent='general_question'\n"
            "4. Simple greetings → intent='greeting'\n"
            "5. Not about diabetes → intent='off_topic'\n\n"
            "Return ONLY valid JSON:\n"
            "{\n"
            '  "intent": "trial_search" | "profile_info" | "general_question" | "greeting" | "off_topic",\n'
            '  "query_type": "trial_query" | "profile_statement" | "knowledge_seeking" | "greeting",\n'
            '  "search_keywords": ["keyword1", "keyword2"],\n'
            '  "is_diabetes_related": true/false,\n'
            '  "user_question": "the question in plain English",\n'
            '  "trial_interest": "what type of trial they want (diet, medication, technology, etc.)"\n'
            "}\n\n"
            "Examples:\n"
            '- \"What trials study liraglutide?\" → intent=\"trial_search\", search_keywords=[\"liraglutide\"]\n'
            "- \"I'm 55 with diabetes\" → intent=\"profile_info\"\n"
            "- \"What is HbA1c?\" → intent=\"general_question\"\n"
        )

        try:
            res = self.model.generate_content(prompt)
            raw = (res.text or "").strip()
            match = re.search(r"\{.*\}", raw, re.DOTALL)
            if match:
                parsed = json.loads(match.group(0))
            else:
                parsed = json.loads(raw)

            # Force trial_search if keywords present
            text_lower = text.lower()
            trial_keywords = [
                "trial", "study", "studies", "research",
                "clinical", "show me", "are there", "what trials"
            ]
            if any(kw in text_lower for kw in trial_keywords):
                parsed["intent"] = "trial_search"
                parsed["query_type"] = "trial_query"

        except Exception:
            # Fallback with keyword detection
            text_lower = text.lower()
            if any(kw in text_lower for kw in ["trial", "study", "research"]):
                parsed = {
                    "intent": "trial_search",
                    "query_type": "trial_query",
                    "search_keywords": [text],
                    "is_diabetes_related": True,
                    "user_question": text,
                    "trial_interest": "general",
                }
            else:
                parsed = {
                    "intent": "general_question",
                    "query_type": "knowledge_seeking",
                    "search_keywords": [],
                    "is_diabetes_related": True,
                    "user_question": text,
                    "trial_interest": None,
                }

        log = log_provenance_step("SymptomParser", text, parsed)
        return parsed, log


# =========================
# PROFILE AGENT
# =========================
class ProfileAgent:
    def __init__(self, initial_profile: Dict[str, Any] = None):
        if initial_profile is None:
            initial_profile = {
                "user_id": "Patient",
                "conditions": ["diabetes"],  # Default context
                "extracted_conditions": [],  # Dynamic memory
                "history": [],
            }
        self.profile = initial_profile

    def update_profile(self, turn_data: Dict[str, Any]):
        """
        Updates history and extracts persistent medical entities.
        """
        self.profile.setdefault("history", []).append(turn_data)
        self.profile.setdefault("extracted_conditions", [])

        parsed = turn_data.get("parsed", {})
        new_symptoms = parsed.get("symptoms", [])

        if new_symptoms:
            current_conditions = set(self.profile["extracted_conditions"])
            for sym in new_symptoms:
                if sym and len(sym) > 3:  # Avoid noise
                    current_conditions.add(sym.lower())
            self.profile["extracted_conditions"] = list(current_conditions)

        snapshot = {
            "user_id": self.profile.get("user_id", "Patient"),
            "known_conditions": self.profile.get("extracted_conditions", []),
            "num_turns": len(self.profile["history"]),
        }
        log = log_provenance_step("ProfileAgent", turn_data, {"profile_snapshot": snapshot})
        return log


# ============================================================
# EVIDENCE-WEIGHTED SCORER (Novel Contribution)
# ============================================================
class EvidenceWeightedScorer:
    """
    Implements evidence-weighted scoring for clinical trials.
    Ranks trials based on multiple quality factors beyond semantic similarity.
    """

    def __init__(self):
        # Status weights (prioritize completed trials for reliable results)
        self.status_weights = {
            "Completed": 1.0,
            "Active, Not Recruiting": 0.9,
            "Recruiting": 0.85,
            "Enrolling By Invitation": 0.8,
            "Not Yet Recruiting": 0.6,
            "Terminated": 0.4,
            "Withdrawn": 0.3,
            "Suspended": 0.3,
            "Unknown Status": 0.5,
        }

        # Study design weights
        self.design_keywords = {
            "randomized controlled": 1.0,
            "double-blind": 0.95,
            "randomized": 0.9,
            "controlled": 0.85,
            "interventional": 0.8,
            "prospective": 0.75,
            "observational": 0.6,
            "retrospective": 0.5,
        }

    def calculate_weighted_score(
        self,
        trial: Dict[str, Any],
        base_confidence: float,
        query: str
    ) -> Dict[str, Any]:
        """
        Calculate evidence-weighted score for a trial.
        Returns weighted_score + breakdown.
        """

        # Factor 1: Base semantic match (35%)
        match_score = base_confidence * 0.35

        # Factor 2: Trial status quality (25%)
        status = str(trial.get("status", "Unknown Status")).strip().title()
        status_score = self.status_weights.get(status, 0.5) * 0.25

        # Factor 3: Study design quality (20%)
        design_score = self._calculate_design_quality(trial) * 0.20

        # Factor 4: Keyword density (10%)
        keyword_score = self._calculate_keyword_density(trial, query) * 0.10

        # Factor 5: Metadata completeness (10%)
        completeness_score = self._calculate_completeness(trial) * 0.10

        # Total weighted score
        weighted_score = (
            match_score +
            status_score +
            design_score +
            keyword_score +
            completeness_score
        )

        breakdown = {
            "base_confidence": base_confidence,
            "weighted_score": weighted_score,
            "factors": {
                "semantic_match": match_score,
                "trial_status": status_score,
                "study_design": design_score,
                "keyword_density": keyword_score,
                "completeness": completeness_score,
            },
        }

        return {
            "weighted_score": min(weighted_score, 1.0),
            "breakdown": breakdown,
        }

    def _calculate_design_quality(self, trial: Dict[str, Any]) -> float:
        text = f"{trial.get('brief_title', '')} {trial.get('brief_summary', '')}".lower()
        max_score = 0.0
        for keyword, weight in self.design_keywords.items():
            if keyword in text:
                max_score = max(max_score, weight)
        # Default moderate score if no keywords found
        return max_score if max_score > 0 else 0.6

    def _calculate_keyword_density(self, trial: Dict[str, Any], query: str) -> float:
        text = f"{trial.get('brief_title', '')} {trial.get('brief_summary', '')}".lower()
        stopwords = {
            "the", "a", "an", "and", "or", "for", "with", "in", "on", "at", "to",
            "of", "is", "are", "what", "trials", "study", "studies"
        }
        query_terms = [
            term for term in query.lower().split()
            if term not in stopwords and len(term) > 2
        ]
        if not query_terms:
            return 0.5
        matches = sum(1 for term in query_terms if term in text)
        density = matches / len(query_terms)
        return min(density, 1.0)

    def _calculate_completeness(self, trial: Dict[str, Any]) -> float:
        fields = [
            "brief_title",
            "brief_summary",
            "eligibility_criteria",
            "interventions",
            "primary_outcomes",
        ]
        filled_fields = 0
        for field in fields:
            value = trial.get(field, "")
            if value and isinstance(value, str) and len(value) > 20:
                filled_fields += 1
        completeness = filled_fields / len(fields)
        return completeness


# ============================================================
# PubMed Helper (NCT → PubMed abstract)
# ============================================================
def fetch_pubmed_abstract_for_nct(nct_id: str):
    """
    Try to find a PubMed article linked to this NCT ID and return its abstract.

    Returns:
        dict { "pmid": str, "abstract": str } or None if not found.
    """
    try:
        # 1) Find PubMed ID via esearch using the NCT ID as Secondary Source ID [si]
        esearch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
        params = {
            "db": "pubmed",
            "term": f"{nct_id}[si]",
            "retmode": "json",
            "retmax": 1,
        }
        r = requests.get(esearch_url, params=params, timeout=10)
        r.raise_for_status()
        data = r.json()
        idlist = data.get("esearchresult", {}).get("idlist", [])
        if not idlist:
            return None

        pmid = idlist[0]

        # 2) Fetch abstract text
        efetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
        params = {
            "db": "pubmed",
            "id": pmid,
            "rettype": "abstract",
            "retmode": "text",
        }
        r2 = requests.get(efetch_url, params=params, timeout=10)
        r2.raise_for_status()
        abstract_text = r2.text.strip()
        if not abstract_text:
            return None

        return {"pmid": pmid, "abstract": abstract_text}
    except Exception:
        # Swallow errors quietly; just means "no abstract found"
        return None


# ============================================================
# RETRIEVAL AGENT
# ============================================================
class RetrievalAgent:
    def __init__(self, embed_model, faiss_index, chunk_map, profile_agent: ProfileAgent = None):
        self.embed_model = embed_model
        self.index = faiss_index
        self.chunk_map = chunk_map
        self.profile_agent = profile_agent
        self.evidence_scorer = EvidenceWeightedScorer()

    def retrieve(self, parsed: Dict[str, Any], top_k: int = 5):
        FETCH_K = top_k * 3

        symptoms = parsed.get("symptoms") or []
        context = parsed.get("context") or ""
        query = parsed.get("user_question") or (" ".join(symptoms) + " " + context).strip()

        if not query:
            retrieval = {"query": "", "trials": [], "avg_confidence": 0.0}
            log = log_provenance_step(
                "RetrievalAgent",
                parsed,
                retrieval,
                {"reason": "empty_query"}
            )
            return retrieval, log

        EXPANSIONS = {
            "insulin": "insulin OR insulin therapy OR insulin treatment OR insulin pump",
            "medication": "medication OR drug OR pharmaceutical OR pharmacological OR treatment",
            "diet": "diet OR dietary OR nutrition OR nutritional OR eating",
            "exercise": "exercise OR physical activity OR fitness OR activity",
            "new": "medication OR drug OR pharmacological OR treatment OR therapy OR intervention",
        }

        query_lower = query.lower()
        for term, expansion in EXPANSIONS.items():
            if term in query_lower:
                query = f"{query} {expansion}"
                break

        # 1. FAISS retrieval
        q_emb = self.embed_model.encode([query])
        distances, indices = self.index.search(q_emb.astype("float32"), FETCH_K)

        initial_candidates = []
        for rank, idx in enumerate(indices[0]):
            if idx == -1:
                continue
            item = self.chunk_map[idx]
            dist = float(distances[0][rank])
            initial_candidates.append({
                "nct_id": item["nct_id"],
                "text": item["text"],
                "status": item["status"],
                "faiss_dist": dist,
            })

        final_trials = []
        confs = []

        # 2. Optional CrossEncoder reranking
        if reranker and initial_candidates:
            pairs = [[query, cand["text"]] for cand in initial_candidates]
            scores = reranker.predict(pairs)

            for i, cand in enumerate(initial_candidates):
                cand["rerank_score"] = float(scores[i])

            initial_candidates.sort(key=lambda x: x["rerank_score"], reverse=True)
            top_hits = initial_candidates[:top_k]

            for rank, item in enumerate(top_hits):
                logit = item["rerank_score"]
                base_conf = 1 / (1 + np.exp(-logit))

                scoring_result = self.evidence_scorer.calculate_weighted_score(
                    trial=item,
                    base_confidence=base_conf,
                    query=query,
                )

                weighted_score = scoring_result["weighted_score"]

                final_trials.append({
                    "nct_id": item["nct_id"],
                    "title": item["text"].split("\n")[0].replace("Title: ", ""),
                    "text": item["text"],
                    "status": item["status"],
                    "confidence": base_conf,
                    "weighted_score": weighted_score,
                    "score_breakdown": scoring_result["breakdown"],
                    "rank": rank + 1,
                    "method": "evidence_weighted",
                })

            final_trials.sort(key=lambda x: x["weighted_score"], reverse=True)
            for i, trial in enumerate(final_trials):
                trial["rank"] = i + 1

            confs = [t["weighted_score"] for t in final_trials]

        else:
            # FAISS-only path
            top_hits = initial_candidates[:top_k]
            for rank, item in enumerate(top_hits):
                base_conf = calculate_confidence_score(item["faiss_dist"])
                scoring_result = self.evidence_scorer.calculate_weighted_score(
                    trial=item,
                    base_confidence=base_conf,
                    query=query,
                )
                final_trials.append({
                    "nct_id": item["nct_id"],
                    "title": item["text"].split("\n")[0].replace("Title: ", ""),
                    "text": item["text"],
                    "status": item["status"],
                    "confidence": base_conf,
                    "weighted_score": scoring_result["weighted_score"],
                    "score_breakdown": scoring_result["breakdown"],
                    "rank": rank + 1,
                    "method": "evidence_weighted_faiss",
                })

            final_trials.sort(key=lambda x: x["weighted_score"], reverse=True)
            for i, trial in enumerate(final_trials):
                trial["rank"] = i + 1
            confs = [t["weighted_score"] for t in final_trials]

        avg_conf = float(np.mean(confs)) if confs else 0.0

        retrieval = {
            "query": query,
            "trials": final_trials,
            "avg_confidence": avg_conf,
        }

        detail = {
            "top_k": top_k,
            "avg_confidence": avg_conf,
            "num_trials": len(final_trials),
            "method": "reranked" if reranker else "faiss_only",
        }

        log = log_provenance_step("RetrievalAgent", parsed, retrieval, detail)
        return retrieval, log


# ============================================================
# DIAGNOSIS / ADVISOR
# ============================================================
class DiagnosisAdvisor:
    def __init__(self, model):
        self.model = model

    def _handle_general_question(self, parsed: Dict[str, Any], retrieved: Dict[str, Any]):
        """Handle general knowledge questions about diabetes."""
        trials = retrieved.get("trials", [])
        user_question = parsed.get("user_question") or " ".join(parsed.get("symptoms", []))

        evidence_parts = []
        for t in trials[:3]:
            evidence_parts.append(f"Trial {t['nct_id']}: {t['text'][:400]}")
        evidence = "\n\n".join(evidence_parts) if evidence_parts else "No specific trials available."

        prompt = (
            "You are a diabetes health educator. Answer the user's question clearly using your medical knowledge.\n"
            "The clinical trial evidence below provides real-world context - mention it if relevant.\n\n"
            f"USER'S QUESTION: {user_question}\n\n"
            "CLINICAL TRIAL CONTEXT (for reference):\n"
            f"{evidence}\n\n"
            "Instructions:\n"
            "- Answer the question directly in 3-5 sentences\n"
            "- Be specific and educational\n"
            "- If trials mention relevant findings, cite them briefly\n"
            "- End with: 'For personalized advice, please consult your healthcare provider.'\n"
        )

        try:
            res = self.model.generate_content(prompt)
            text = (res.text or "").strip()
            if not text or len(text) < 50:
                text = (
                    "I don't have enough information to answer this question accurately. "
                    "Please consult your healthcare provider."
                )
            return text
        except Exception:
            return "Unable to generate an answer at this time. Please try rephrasing your question."

    def _handle_symptom_query(
        self,
        parsed: Dict[str, Any],
        retrieved: Dict[str, Any],
        profile: Dict[str, Any],
    ):
        """
        Generate response for diabetes clinical trial search queries with
        readable paragraph summaries and PubMed abstracts when available.
        """
        trials = retrieved.get("trials", [])
        user_input = parsed.get("user_question", "")

        if not trials:
            return "No relevant trials found. Please try refining your query."

        formatted_trials = []
        for t in trials[:5]:
            title = t["text"].split("\n")[0].replace("Title: ", "")
            status = t.get("status", "Unknown")
            weighted_score = t.get("weighted_score", 0)

            # Extract the ClinicalTrials.gov summary text
            raw_text = t.get("text", "")
            brief_summary = raw_text.split("Summary:", 1)[-1].strip()

            if brief_summary:
                # Ask Gemini to turn the CT.gov summary into a short paragraph
                prompt = (
                    "Rewrite the following clinical trial summary as a short, clear paragraph "
                    "about what the study is testing regarding diabetes:\n\n"
                    f"{brief_summary}\n\n"
                    "Guidelines:\n"
                    "- Use 2–4 sentences\n"
                    "- Plain English\n"
                    "- Include purpose and who it’s for\n"
                    "- Minimize jargon\n"
                )
                try:
                    res = self.model.generate_content(prompt)
                    brief_summary = res.text.strip() if res.text else brief_summary
                except Exception:
                    if len(brief_summary) > 600:
                        brief_summary = brief_summary[:600] + "..."
            else:
                brief_summary = "No summary available."

            # NEW: PubMed abstract lookup (raw, not summarized)
            pubmed_block = ""
            pub = fetch_pubmed_abstract_for_nct(t["nct_id"])
            if pub:
                abs_text = pub["abstract"]
                # Limit length so the UI doesn't explode; still show most of it
                max_len = 2000
                if len(abs_text) > max_len:
                    abs_text = abs_text[:max_len] + "..."
                pubmed_block = (
                    f"  PubMed abstract (PMID {pub['pmid']}):\n"
                    f"  {abs_text}\n\n"
                    f"  PubMed link: https://pubmed.ncbi.nlm.nih.gov/{pub['pmid']}/\n\n"
                )

            formatted_trials.append(
                f"**{t['nct_id']}** (Relevance: {weighted_score:.0%})\n"
                f"• {title}\n"
                f"  Status: {status}\n\n"
                f"  {brief_summary}\n\n"
                f"{pubmed_block}"
            )

        trials_text = "\n\n".join(formatted_trials)
        num_trials = len(formatted_trials)

        response = (
            f"I found {num_trials} diabetes clinical trial"
            f"{'s' if num_trials != 1 else ''} relevant to your request:\n\n"
            f"{trials_text}\n\n"
            "Summary: These trials explore diabetes treatments and management. "
            "More details are available using the listed NCT IDs.\n\n"
            "To learn more or participate, visit clinicaltrials.gov and search by NCT ID. "
            "Discuss with your healthcare provider before enrolling."
        )

        return response

    def advise(self, parsed: Dict[str, Any], retrieved: Dict[str, Any], profile: Dict[str, Any]):
        trials = retrieved.get("trials", [])
        avg_conf = retrieved.get("avg_confidence", 0.0)
        query_type = parsed.get("query_type", "symptom_matching")
        is_diabetes_related = parsed.get("is_diabetes_related", True)

        draft = {
            "recommendation": "",
            "avg_confidence": avg_conf,
            "query_type": query_type,
        }

        if not is_diabetes_related:
            draft["recommendation"] = (
                "I'm specialized in diabetes-related clinical trials. Your query appears to be "
                "about symptoms or conditions not directly related to diabetes. "
                "If you have diabetes-related questions or symptoms (like high blood sugar, "
                "insulin management, complications, etc.), I'd be happy to help! "
                "Otherwise, please consult your healthcare provider for your current symptoms."
            )
            draft["confidence_veto"] = True
            log = log_provenance_step(
                "DiagnosisAdvisor",
                parsed,
                draft,
                {"veto": True, "reason": "off_topic"},
            )
            return draft, log

        if not trials or avg_conf < 0.05:
            draft["recommendation"] = (
                "EVIDENCE IS INSUFFICIENT TO ANSWER THIS QUESTION DIRECTLY based on the "
                "retrieved clinical trials. Please consult your healthcare provider."
            )
            draft["confidence_veto"] = True
            log = log_provenance_step(
                "DiagnosisAdvisor",
                parsed,
                draft,
                {"veto": True, "reason": "low_confidence"},
            )
            return draft, log

        if query_type == "knowledge_seeking":
            draft["recommendation"] = self._handle_general_question(parsed, retrieved)
        else:
            draft["recommendation"] = self._handle_symptom_query(parsed, retrieved, profile)

        draft["confidence_veto"] = False
        log = log_provenance_step("DiagnosisAdvisor", parsed, draft)
        return draft, log


# ============================================================
# SAFETY FILTER
# ============================================================
class ActiveSafetyFilter:
    def __init__(self, model):
        self.model = model
        self.safety_cfg = {
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        }

    def verify(self, advice_text: str, trials: List[Dict[str, Any]]):
        # Skip safety check for trial listing-style responses
        if any(marker in advice_text for marker in ["NCT", "clinical trial", "clinicaltrials.gov"]):
            log = log_provenance_step(
                "ActiveSafetyFilter",
                {"advice": advice_text},
                {"final_text": advice_text, "status": "Pass (Trial Listing)"},
            )
            return advice_text, "Pass (Trial Listing)", log

        evidence_text = "\n".join(t["text"][:500] for t in trials[:3])

        audit_prompt = (
            "You are a Medical Safety Officer reviewing AI-generated advice.\n\n"
            "ADVICE:\n"
            f"{advice_text}\n\n"
            "EVIDENCE FROM CLINICAL TRIALS:\n"
            f"{evidence_text}\n\n"
            "Check for safety issues:\n"
            "- If the advice suggests stopping or changing medication without a doctor → UNSAFE.\n"
            "- If it gives a diagnosis → UNSAFE.\n"
            "- If it makes claims not supported by the evidence → UNSAFE.\n"
            "- If it just lists clinical trials with disclaimers → SAFE.\n\n"
            "If the advice is acceptable, respond with exactly: SAFE\n"
            "If it is not acceptable, respond starting with: CORRECTED: <safer version>\n"
        )

        try:
            res = self.model.generate_content(audit_prompt, safety_settings=self.safety_cfg)
            txt = (res.text or "").strip()
            if txt.startswith("SAFE") or "SAFE" in txt:
                final_text = advice_text
                status = "Pass"
            else:
                final_text = f"⚠️ SAFETY REVISION:\n{txt}"
                status = "Revised"
        except Exception:
            if "NCT" in advice_text or "clinical trial" in advice_text.lower():
                final_text = advice_text
                status = "Pass (API Fallback)"
            else:
                final_text = "⚠️ Safety filter triggered. Please consult a doctor."
                status = "Revised (API Error)"

        log = log_provenance_step(
            "ActiveSafetyFilter",
            {"advice": advice_text},
            {"final_text": final_text, "status": status},
        )
        return final_text, status, log


# ============================================================
# HEALTHCARE BOT (Orchestrator)
# ============================================================
class HealthcareBot:
    def __init__(self, gemini_model, embed_model, faiss_index, chunk_map, initial_profile=None):
        self.parser = SymptomParser(gemini_model)
        self.profile_agent = ProfileAgent(initial_profile)
        self.retriever = RetrievalAgent(embed_model, faiss_index, chunk_map, self.profile_agent)
        self.advisor = DiagnosisAdvisor(gemini_model)
        self.safety = ActiveSafetyFilter(gemini_model)

        self.history: List[Dict[str, Any]] = []
        self.provenance_chain: List[Dict[str, Any]] = []

    def _handle_simple_greeting(self, user_input: str):
        user_id = self.profile_agent.profile.get("user_id", "there")
        msg = (
            f"Hello {user_id}! I'm your **Clinical Trial Research Assistant** for diabetes. 🔬\n\n"
            "I can help you find relevant diabetes clinical trials from a database of **22,000+ studies**.\n\n"
            "**Try asking:**\n"
            "- 'What trials are studying insulin therapy?'\n"
            "- 'Show me trials about low-carb diets'\n"
            "- 'Are there trials testing new medications?'\n"
            "- 'I'm 55 with type 2 diabetes, what trials can I join?'\n\n"
            "I search real trial data from ClinicalTrials.gov. How can I help you explore diabetes research today?"
        )

        log = log_provenance_step("GreetingAgent", user_input, msg, {"type": "greeting"})
        self.provenance_chain.append(log)

        session_hash = generate_reproducibility_hash(self.history + [{"query": user_input}])
        self.history.append({"query": user_input, "response_hash": session_hash})

        return {
            "recommendation": msg,
            "cited_trials": [],
            "safety_status": "Non-RAG",
            "session_hash": session_hash,
            "provenance_chain": self.provenance_chain,
        }

    def _handle_off_topic(self, user_input: str, parsed: Dict[str, Any]):
        msg = (
            "I'm specialized in diabetes-related clinical trials. Your query appears to be "
            "about symptoms or conditions not directly related to diabetes. "
            "If you have diabetes-related questions, I'd be happy to help!"
        )
        log = log_provenance_step("OffTopicHandler", user_input, msg, {"type": "off_topic"})
        self.provenance_chain.append(log)
        session_hash = generate_reproducibility_hash(self.history + [{"query": user_input}])

        return {
            "recommendation": msg,
            "cited_trials": [],
            "safety_status": "Off-topic",
            "session_hash": session_hash,
            "provenance_chain": self.provenance_chain,
        }

    def _handle_knowledge_question(self, user_input: str, parsed: Dict[str, Any]):
        user_question = parsed.get("user_question", user_input)
        prompt = (
            "You are a certified diabetes educator. Answer this question clearly and accurately.\n"
            f"QUESTION: {user_question}\n"
        )
        try:
            res = self.advisor.model.generate_content(prompt)
            answer = (res.text or "").strip()
        except Exception:
            answer = "Unable to answer at this time."

        log = log_provenance_step("KnowledgeAgent", user_input, answer, {"type": "general_knowledge"})
        self.provenance_chain.append(log)
        session_hash = generate_reproducibility_hash(self.history + [{"query": user_input}])

        return {
            "recommendation": answer,
            "cited_trials": [],
            "safety_status": "Knowledge-Based",
            "session_hash": session_hash,
            "provenance_chain": self.provenance_chain,
        }

    def _handle_generic_trial_query(self, user_input: str, parsed: Dict[str, Any]):
        """Handle generic queries that need more specificity."""
        msg = (
            "I found that question a bit broad. I have 22,000+ diabetes trials in my database. "
            "To help you better, could you specify:\n\n"
            "**Drug/Medication Trials:**\n"
            "- Specific drugs: 'trials testing metformin', 'liraglutide trials'\n"
            "- Drug classes: 'GLP-1 trials', 'SGLT2 inhibitor trials'\n"
            "- Insulin: 'insulin pump trials', 'insulin therapy trials'\n\n"
            "**Lifestyle Trials:**\n"
            "- Diet: 'low-carb diet trials', 'Mediterranean diet trials'\n"
            "- Exercise: 'physical activity trials', 'exercise trials'\n\n"
            "**Technology Trials:**\n"
            "- Monitoring: 'CGM trials', 'glucose monitoring trials'\n"
            "- Apps: 'diabetes app trials', 'digital health trials'\n\n"
            "**Or describe your situation:**\n"
            "- 'I'm 55 with type 2 diabetes, what trials can I join?'\n"
            "- 'Trials for managing high blood sugar'\n\n"
            "What would you like to explore?"
        )

        log = log_provenance_step("GenericQueryHandler", user_input, msg, {"type": "needs_refinement"})
        self.provenance_chain.append(log)

        session_hash = generate_reproducibility_hash(self.history + [{"query": user_input}])
        self.history.append({"query": user_input, "response_hash": session_hash})

        return {
            "recommendation": msg,
            "cited_trials": [],
            "safety_status": "Refinement Needed",
            "session_hash": session_hash,
            "provenance_chain": self.provenance_chain,
        }

    def process_query(self, user_input: str):
        self.provenance_chain = []

        # 1. Parse
        parsed, parse_log = self.parser.parse(user_input)
        self.provenance_chain.append(parse_log)

        intent = (parsed.get("intent") or "trial_search").lower()
        query_type = parsed.get("query_type", "trial_query")
        is_diabetes_related = parsed.get("is_diabetes_related", True)

        # Greetings
        if intent == "greeting":
            return self._handle_simple_greeting(user_input)

        # Off-topic
        if intent == "off_topic" or not is_diabetes_related:
            return self._handle_off_topic(user_input, parsed)

        # Profile info
        if intent == "profile_info":
            msg = (
                "Thank you for sharing your information. I've noted your details. "
                "What type of clinical trials would you like to explore? "
                "For example: 'Show me trials about diet management' or 'What trials test new medications?'"
            )
            log = log_provenance_step("ProfileAgent", user_input, msg, {"action": "profile_stored"})
            self.provenance_chain.append(log)

            session_hash = generate_reproducibility_hash(self.history + [{"query": user_input}])
            return {
                "recommendation": msg,
                "cited_trials": [],
                "safety_status": "Profile Update",
                "session_hash": session_hash,
                "provenance_chain": self.provenance_chain,
            }

        # Pure education (no trial search)
        if intent == "general_question" and query_type == "knowledge_seeking":
            if "trial" not in user_input.lower() and "study" not in user_input.lower():
                return self._handle_knowledge_question(user_input, parsed)

        # DEFAULT: trial search
        retrieved, retrieve_log = self.retriever.retrieve(parsed)
        self.provenance_chain.append(retrieve_log)

        # Generic query detection
        generic_terms = ["new", "any", "some", "recent", "latest", "medications", "trials", "studies"]
        is_generic = sum(1 for term in generic_terms if term in user_input.lower()) >= 2

        trials = retrieved.get("trials", [])
        avg_conf = retrieved.get("avg_confidence", 0.0)
        top_score = trials[0]["weighted_score"] if trials else 0.0

        if is_generic and (avg_conf < 0.50 or top_score < 0.55):
            return self._handle_generic_trial_query(user_input, parsed)

        # 3. Advisor
        draft_advice, advise_log = self.advisor.advise(parsed, retrieved, self.profile_agent.profile)
        self.provenance_chain.append(advise_log)

        trials = retrieved.get("trials", [])
        if draft_advice.get("confidence_veto", False) or not trials:
            final_text = draft_advice["recommendation"]
            safety_status = "Vetoed (Low Confidence)"
            evidence_list = []
        else:
            final_text, safety_status, safety_log = self.safety.verify(draft_advice["recommendation"], trials)
            self.provenance_chain.append(safety_log)
            evidence_list = trials

        nct_ids = [t["nct_id"] for t in evidence_list]
        session_hash = generate_reproducibility_hash(self.history + [{"query": user_input}])

        # Update profile/history
        turn_data = {
            "query": user_input,
            "parsed": parsed,
            "nct_ids": nct_ids,
            "safety_status": safety_status,
            "session_hash": session_hash,
        }
        profile_log = self.profile_agent.update_profile(turn_data)
        self.provenance_chain.append(profile_log)
        self.history.append({"query": user_input, "response_hash": session_hash})

        return {
            "recommendation": final_text,
            "cited_trials": nct_ids,
            "safety_status": safety_status,
            "session_hash": session_hash,
            "provenance_chain": self.provenance_chain,
        }


# ============================================================
# GLOBAL BOT INSTANCE + ENTRYPOINT
# ============================================================
default_profile = {
    "user_id": "Patient",
    "conditions": ["diabetes"],
    "extracted_conditions": [],
}

_bot = HealthcareBot(gemini_model, embed_model, faiss_index, chunk_map, initial_profile=default_profile)

def run_bot(user_input: str) -> Dict[str, Any]:
    return _bot.process_query(user_input)


Overwriting run_bot.py


UI frontend application simple web interface

https://docs.streamlit.io/develop/tutorials/chat-and-llm-apps/build-conversational-apps

In [21]:
%%writefile app.py
import streamlit as st
import os

if "GEMINI_API_KEY" not in os.environ:
    st.error("⚠️ API Key missing! Please run the 'Secure Input' cell in the notebook first.")

from run_bot import run_bot

st.title("Clinical Trial Health Advisor 🤖")
st.caption("AI for Healthcare - Clinical Trials RAG")

if "messages" not in st.session_state:
    st.session_state.messages = []

for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

if user_input := st.chat_input("Describe your symptoms..."):
    st.session_state.messages.append({"role": "user", "content": user_input})
    with st.chat_message("user"):
        st.markdown(user_input)

    with st.spinner("Searching clinical trials..."):
        result = run_bot(user_input)
        reply = result["recommendation"]

    with st.chat_message("assistant"):
        st.markdown(reply)

    st.session_state.messages.append({"role": "assistant", "content": reply})

Overwriting app.py


In [9]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!mv cloudflared-linux-amd64 cloudflared
!chmod +x cloudflared

In [22]:
#AI LLM
!streamlit run app.py &>/dev/null&
!./cloudflared tunnel --url http://localhost:8501 --no-autoupdate

2025-11-27T07:07:11Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-27T07:07:11Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-27T07:07:17Z INF +--------------------------------------------------------------------------------------------+
2025-11-27T07:07:17Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-27T07:07:17Z INF |  https://colored-cooler-chicago-admitted.trycloudflare

In [20]:
from run_bot import run_bot

print(run_bot("What trials are studying insulin therapy?"))


{'recommendation': 'I found 5 diabetes clinical trials relevant to your request:\n\n**NCT00115973** (Relevance: 71%)\n• A Study of the Treatment of Type 2 Diabetes With an Insulin Infusion Pump\n  Status: Completed\n\n  This trial is conducted in the United States of America (USA). This is an in-patient trial investigating stepwise dose increase in a period of up to 3-weeks followed by a 10-week out-patient maintenance period. A telephone contact visit is scheduled as a follow-up for the final clinic visit. A subject\'s participation in this trial would be expected to be up to 16 weeks.\n\n**NCT04981808** (Relevance: 71%)\n• Diabetes teleMonitoring of Patients in Insulin Therapy\n  Status: Completed\n\n  The trial is an open-label randomized controlled trial. Patients with T2D on insulin therapy will be randomized to a telemonitoring group (intervention) and a usual care group (control). The telemonitoring group will use various devices at home. Hospital staff will monitor their data f